By [Yulandy Chiu](https://www.youtube.com/@YulandySpace)

Aided with Gemini/Claude/ChatGPT and modified by Yulandy Chiu

Version: 2024/12/24

Video: [RAG implementation using Google Search and Gemini](https://youtu.be/cmHsOdlieXY)

 This code is licensed under the Creative Commons Attribution-NonCommercial 4.0
 International License (CC BY-NC 4.0). You are free to use, modify, and share this code for non-commercial purposes, provided you give appropriate credit. For more details, see the LICENSE file or visit: https://creativecommons.org/licenses/by-nc/4.0/
 © [2024] [Yulandy Chiu](https://www.youtube.com/@YulandySpace)

In [ ]:
# 測試網路搜尋功能
import google.generativeai as genai
from google.colab import userdata

api_key=userdata.get('Gemini_API_Key')
genai.configure(api_key=api_key)
model = genai.GenerativeModel("gemini-1.5-flash")
response = model.generate_content("Explain how AI works")

# response = model.generate_content("提供台灣2024年的棒球新聞")
print(response.text)

In [ ]:
# Step 1: 擷取需要RAG的 query

user_query="使用繁體中文回答。根據2024年台灣棒球新聞，產生長篇故事"


import google.generativeai as genai

def generate_gemini_response(llm_input, api_key, model_name):
    """
    使用 Google Gemini API 生成回應。

    Args:
        llm_input: 用戶輸入（包含問題和檢索到的資料）。
        api_key: Google Gemini API 金鑰。
        model_name: Gemini 模型名稱，預設是 "gemini-1.5-flash"。
    Returns:
        Gemini 生成的回應。
    """
    genai.configure(api_key=api_key)
    model = genai.GenerativeModel(model_name=model_name)
    try:
        response = model.generate_content(llm_input)
        return response.text
    except Exception as e:
        print(f"Error during Gemini response generation: {e}")
        return None

# 參數修改
from google.colab import userdata
api_key=userdata.get('Gemini_API_Key')

llm_input = f"根據要求，分析需要上網查詢的資訊(內部知識不足夠)。生成需要上網查詢資訊的的關鍵字(不需要相關說明，回傳資料可以直接複製、貼上使用): {user_query} " # 控制输入长度在4000字符以内
model_name="gemini-1.5-flash"
Google_Search_Keywords = generate_gemini_response(llm_input, api_key, model_name)
print(Google_Search_Keywords)

In [ ]:
# Step 2: 檢索 Retrievel (Google Search)
!pip install google-search-results

from googleapiclient.discovery import build

def search_google_custom(query, api_key, cse_id, num_results):
    """
    使用 Google Custom Search API 進行搜尋。

    Args:
        query: 用戶的搜尋查詢。
        api_key: Google API 金鑰。
        cse_id: Google Custom Search Engine 的 ID。
        num_results: 返回的搜尋結果數量。

    Returns:
        搜尋結果的列表，每個結果包含 title、link 和 snippet。
    """
    service = build("customsearch", "v1", developerKey=api_key)
    try:
      response = (
          service.cse()
          .list(q=query, cx=cse_id, num=num_results)
          .execute()
      )
      results = response.get("items", []) # 使用 get 方法避免当 items 不存在时报错

      formatted_results = []
      for item in results:
          formatted_results.append({
              "title": item.get("title", ""),
              "link": item.get("link", ""),
              "snippet": item.get("snippet", "")
          })
      return formatted_results
    except Exception as e:
        print(f"Error during Google Custom Search: {e}")
        return []


# 參數修改
from google.colab import userdata

api_key=userdata.get('Google_Search_API_Key')
cse_id =userdata.get('Google_CSE_ID')
query=Google_Search_Keywords
search_results = search_google_custom(query, api_key, cse_id, num_results=5)

if search_results:
    for result in search_results:
        print(f"Title: {result['title']}")
        print(f"Link: {result['link']}")
        print(f"Snippet: {result['snippet']}")
        print("-" * 30)
else:
    print("No search results found.")




 # 建立 LLM 輸入字串
Retrieval_content = "這是網路上搜尋到的資料可供參考:\n"
for i, result in enumerate(search_results):
  Retrieval_content += f"標題：{result['title']}\n"
  Retrieval_content += f"摘要：{result['snippet']}\n"
print(Retrieval_content)


In [ ]:
# Step 3 增加生成 augmented generation

# augmented
llm_input = f"Prompt:\n"+ f"{user_query}\n" + f"{Retrieval_content[:4000]}" # 控制输入长度在4000字符以内
print(llm_input)

# generaiton
gemini_output = generate_gemini_response(llm_input, api_key, model_name)

print(f"Gemini輸出:\n" +gemini_output)

In [ ]:
# 完整Rag實現
%reset -sf
user_query="使用繁體中文回答。根據2024年Donald Trump當選美國總統的新聞，產生長篇故事"


import google.generativeai as genai

def generate_gemini_response(llm_input, api_key, model_name):
    """
    使用 Google Gemini API 生成回應。

    Args:
        llm_input: 用戶輸入（包含問題和檢索到的資料）。
        api_key: Google Gemini API 金鑰。
        model_name: Gemini 模型名稱，預設是 "gemini-1.5-flash"。
    Returns:
        Gemini 生成的回應。
    """
    genai.configure(api_key=api_key)
    model = genai.GenerativeModel(model_name=model_name)
    try:
        response = model.generate_content(llm_input)
        return response.text
    except Exception as e:
        print(f"Error during Gemini response generation: {e}")
        return None

# 參數修改
from google.colab import userdata
api_key=userdata.get('Gemini_API_Key')

llm_input = f"根據要求，分析需要上網查詢的資訊(內部知識不足夠)。生成需要上網查詢資訊的的關鍵字(不需要相關說明，回傳資料可以直接複製、貼上使用): {user_query} " # 控制输入长度在4000字符以内
model_name="gemini-1.5-flash"
Google_Search_Keywords = generate_gemini_response(llm_input, api_key, model_name)
# print(Google_Search_Keywords)

# Step 2: 檢索 Retrievel (Google Search)
# !pip install google-search-results

from googleapiclient.discovery import build

def search_google_custom(query, api_key, cse_id, num_results):
    """
    使用 Google Custom Search API 進行搜尋。

    Args:
        query: 用戶的搜尋查詢。
        api_key: Google API 金鑰。
        cse_id: Google Custom Search Engine 的 ID。
        num_results: 返回的搜尋結果數量。

    Returns:
        搜尋結果的列表，每個結果包含 title、link 和 snippet。
    """
    service = build("customsearch", "v1", developerKey=api_key)
    try:
      response = (
          service.cse()
          .list(q=query, cx=cse_id, num=num_results)
          .execute()
      )
      results = response.get("items", []) # 使用 get 方法避免当 items 不存在时报错

      formatted_results = []
      for item in results:
          formatted_results.append({
              "title": item.get("title", ""),
              "link": item.get("link", ""),
              "snippet": item.get("snippet", "")
          })
      return formatted_results
    except Exception as e:
        print(f"Error during Google Custom Search: {e}")
        return []


# 參數修改
from google.colab import userdata

api_key=userdata.get('Google_Search_API_Key')
cse_id =userdata.get('Google_CSE_ID')
query=Google_Search_Keywords
search_results = search_google_custom(query, api_key, cse_id, num_results=5)

# if search_results:
#     for result in search_results:
#         print(f"Title: {result['title']}")
#         print(f"Link: {result['link']}")
#         print(f"Snippet: {result['snippet']}")
#         print("-" * 30)
# else:
#     print("No search results found.")




 # 建立 LLM 輸入字串
Retrieval_content = "這是網路上搜尋到的資料可供參考:\n"
for i, result in enumerate(search_results):
  Retrieval_content += f"標題：{result['title']}\n"
  Retrieval_content += f"摘要：{result['snippet']}\n"
# print(Retrieval_content)

# Step 3 增加生成 augmented generation

# augmented
llm_input = f"Prompt:\n"+ f"{user_query}\n" + f"{Retrieval_content[:4000]}" # 控制输入长度在4000字符以内
# print(llm_input)

# generaiton
gemini_output = generate_gemini_response(llm_input, api_key, model_name)

print(f"Gemini輸出:\n" +gemini_output)